In [1]:
from pyvis import network as net
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

In [2]:
def create_funnel_graph(df, chart_title='Funnel graph', source='source', target='target', value='user'):
    all_nodes = df[source].values.tolist() + df[target].values.tolist()

    source_indices = [all_nodes.index(source) for source in df[source]]
    target_indices = [all_nodes.index(target) for target in df[target]]

    colors = px.colors.qualitative.Plotly

    node_colors_mappings = dict([(node, np.random.choice(colors)) for node in all_nodes])

    node_colors = [node_colors_mappings[node] for node in all_nodes]
    edge_colors = [node_colors_mappings[node] for node in df[source]]

    fig = go.Figure(data=[
        go.Sankey(
            node=dict(pad=20,
                    thickness=20,
                    line=dict(color='black', width=1.0),
                    label=all_nodes,
                    color=node_colors),
            link=dict(source=source_indices,
                    target=target_indices,
                    value=df[value],
                    color=edge_colors)
        )
    ])

    fig.update_layout(title_text=chart_title,
                    height=1000,
                    font=dict(size=10, color='white'),
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='grey')

    return fig


# First play action

In [3]:
df = pd.read_csv('data/2nd_play_short.csv')
df = df.dropna()

total_user = df[df['source']=='client_enter_game-2']['user'].sum()
df = df.groupby(['source', 'target'])['user'].sum().reset_index().rename(columns={0:'user'})
df['user'] = df.apply(lambda x: round(x['user'] / total_user * 100, 1), axis=1)

fig = create_funnel_graph(df, chart_title='2nd to 3rd play drop')

fig.show()

In [4]:
fig.write_html("./result_graph/9min_drop_play3.html")

# Purchase value graph

In [5]:
df_purchase = pd.read_csv('./data/purchase.csv')
df_purchase = df_purchase[(df_purchase['purchase_count']<=2)]
# df['target'].fillna('EXIT', inplace=True)
df_purchase = df_purchase.dropna()
# df = df.sort_values("user", ascending=True)
# df['pct'] = df.apply(lambda x: x['user'] / df.groupby(['source'])['user'].sum()[x['source']], axis=1)
df_purchase.head()

,purchase_count,source,target,user
0,1,1. $1.99,2. $0.99,21
3,1,1. $19.99,2. $9.99,1
4,1,1. $0.99,2. $1.99,8
5,1,1. $4.99,2. $1.99,3
6,1,1. $9.99,2. $6.99,3


In [6]:
fig = create_funnel_graph(df_purchase, chart_title="Purchase funnel graph")

fig.show()